In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "7a765629",
   "metadata": {},
   "source": [
    "# Semantic Search using the Inference API with the OpenAI service\n",
    "\n",
    "[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/07-inference.ipynb)\n",
    "\n",
    "\n",
    "Learn how to use the [Inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-apis.html) with the OpenAI service for semantic search."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f9101eb9",
   "metadata": {},
   "source": [
    "# 🧰 Requirements\n",
    "\n",
    "For this example, you will need:\n",
    "\n",
    "- An Elastic deployment:\n",
    "   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook))\n",
    "\n",
    "- Elasticsearch 8.12 or above.\n",
    "   \n",
    "- A paid [OpenAI account](https://openai.com/) is required to use the Inference API with \n",
    "the OpenAI service as the OpenAI free trial API usage is limited. "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4cd69cc0",
   "metadata": {},
   "source": [
    "# Create Elastic Cloud deployment\n",
    "\n",
    "If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f27dffbf",
   "metadata": {},
   "source": [
    "# Install packages and connect with Elasticsearch Client\n",
    "\n",
    "To get started, we'll need to connect to our Elastic deployment using the Python client (version 8.12.0 or above).\n",
    "Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.\n",
    "\n",
    "First we need to `pip` install the following packages:\n",
    "\n",
    "- `elasticsearch`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8c4b16bc",
   "metadata": {},
   "outputs": [],
   "source": [
    "!pip install elasticsearch"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "41ef96b3",
   "metadata": {},
   "source": [
    "Next, we need to import the modules we need. 🔐 NOTE: getpass enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "690ff9af",
   "metadata": {},
   "outputs": [],
   "source": [
    "from elasticsearch import Elasticsearch, helpers, exceptions\n",
    "from urllib.request import urlopen\n",
    "from getpass import getpass\n",
    "import json\n",
    "import time"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "23fa2b6c",
   "metadata": {},
   "source": [
    "Now we can instantiate the Python Elasticsearch client.\n",
    "\n",
    "First we prompt the user for their password and Cloud ID.\n",
    "Then we create a `client` object that instantiates an instance of the `Elasticsearch` class."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "195cc597",
   "metadata": {},
   "outputs": [],
   "source": [
    "# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id\n",
    "ELASTIC_CLOUD_ID = getpass(\"Elastic Cloud ID: \")\n",
    "\n",
    "# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key\n",
    "ELASTIC_API_KEY = getpass(\"Elastic Api Key: \")\n",
    "\n",
    "# Create the client instance\n",
    "client = Elasticsearch(\n",
    "    # For local development\n",
    "    # hosts=[\"http://localhost:9200\"]\n",
    "    cloud_id=ELASTIC_CLOUD_ID,\n",
    "    api_key=ELASTIC_API_KEY,\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "03e377d9-be69-42c8-9d1a-d6019f7690e7",
   "metadata": {},
   "source": [
    "### Enable Telemetry\n",
    "\n",
    "Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See [telemetry.py](https://github.com/elastic/elasticsearch-labs/blob/main/telemetry/telemetry.py) for details. Thank you!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a6825796-679b-40e9-8253-a5c0dac43429",
   "metadata": {},
   "outputs": [],
   "source": [
    "!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py\n",
    "from telemetry import enable_telemetry\n",
    "\n",
    "client = enable_telemetry(client, \"07-inference\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b1115ffb",
   "metadata": {},
   "source": [
    "### Test the Client\n",
    "Before you continue, confirm that the client has connected with this test."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "cc0de5ea",
   "metadata": {},
   "outputs": [],
   "source": [
    "print(client.info())"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "659c5890",
   "metadata": {},
   "source": [
    "Refer to [the documentation](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect to a self-managed deployment.\n",
    "\n",
    "Read [this page](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect using API keys."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "840d92f0",
   "metadata": {},
   "source": [
    "<a name=\"create-the-inference-endpoint\"></a>\n",
    "## Create the inference endpoint\n",
    "\n",
    "Let's create the inference endpoint by using the [Create inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html).\n",
    "\n",
    "You'll need an OpenAI API key for this that you can find in your OpenAI account under the [API keys section](https://platform.openai.com/api-keys). A paid membership is required to complete the steps in this notebook as the OpenAI free trial API usage is limited."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "0d007737",
   "metadata": {},
   "outputs": [],
   "source": [
    "API_KEY = getpass(\"OpenAI API key:  \")\n",
    "\n",
    "client.inference.put_model(\n",
    "    task_type=\"text_embedding\",\n",
    "    inference_id=\"my_openai_embedding_model\",\n",
    "    body={\n",
    "        \"service\": \"openai\",\n",
    "        \"service_settings\": {\"api_key\": API_KEY},\n",
    "        \"task_settings\": {\"model\": \"text-embedding-ada-002\"},\n",
    "    },\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1f2e48b7",
   "metadata": {},
   "source": [
    "**NOTE:** If you use Elasticsearch 8.12, you must change `inference_id` in the snippet above to `model_id`! "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1024d070",
   "metadata": {},
   "source": [
    "## Create an ingest pipeline with an inference processor\n",
    "\n",
    "Create an ingest pipeline with an inference processor by using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method. Reference the inference endpoint created above as `model_id` to infer against the data that is being ingested in the pipeline."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "6ace9e2e",
   "metadata": {},
   "outputs": [],
   "source": [
    "client.ingest.put_pipeline(\n",
    "    id=\"openai_embeddings_pipeline\",\n",
    "    description=\"Ingest pipeline for OpenAI inference.\",\n",
    "    processors=[\n",
    "        {\n",
    "            \"inference\": {\n",
    "                \"model_id\": \"my_openai_embedding_model\",\n",
    "                \"input_output\": {\n",
    "                    \"input_field\": \"plot\",\n",
    "                    \"output_field\": \"plot_embedding\",\n",
    "                },\n",
    "            }\n",
    "        }\n",
    "    ],\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "76d07567",
   "metadata": {},
   "source": [
    "Let's note a few important parameters from that API call:\n",
    "\n",
    "- `inference`: A processor that performs inference using a machine learning model.\n",
    "- `model_id`: Specifies the ID of the inference endpoint to be used. In this example, the inference ID is set to `my_openai_embedding_model`. Use the inference ID you defined when created the inference task.\n",
    "- `input_output`: Specifies input and output fields.\n",
    "- `input_field`: Field name from which the `dense_vector` representation is created.\n",
    "- `output_field`:  Field name which contains inference results. "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "28e12d7a",
   "metadata": {},
   "source": [
    "## Create index\n",
    "\n",
    "The mapping of the destination index - the index that contains the embeddings that the model will create based on your input text - must be created. The destination index must have a field with the [dense_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html) field type to index the output of the OpenAI model.\n",
    "\n",
    "Let's create an index named `openai-movie-embeddings` with the mappings we need."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "6ddcbca3",
   "metadata": {},
   "outputs": [],
   "source": [
    "client.indices.delete(index=\"openai-movie-embeddings\", ignore_unavailable=True)\n",
    "client.indices.create(\n",
    "    index=\"openai-movie-embeddings\",\n",
    "    settings={\"index\": {\"default_pipeline\": \"openai_embeddings_pipeline\"}},\n",
    "    mappings={\n",
    "        \"properties\": {\n",
    "            \"plot_embedding\": {\n",
    "                \"type\": \"dense_vector\",\n",
    "                \"dims\": 1536,\n",
    "                \"similarity\": \"dot_product\",\n",
    "            },\n",
    "            \"plot\": {\"type\": \"text\"},\n",
    "        }\n",
    "    },\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "07c187a9",
   "metadata": {},
   "source": [
    "## Insert Documents\n",
    "\n",
    "Let's insert our example dataset of 12 movies. You need a paid OpenAI account to complete this step, otherwise the documentation ingest will time out due to the API request rate limits."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d68737cb",
   "metadata": {},
   "outputs": [],
   "source": [
    "url = \"https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json\"\n",
    "response = urlopen(url)\n",
    "\n",
    "# Load the response data into a JSON object\n",
    "data_json = json.loads(response.read())\n",
    "\n",
    "# Prepare the documents to be indexed\n",
    "documents = []\n",
    "for doc in data_json:\n",
    "    documents.append(\n",
    "        {\n",
    "            \"_index\": \"openai-movie-embeddings\",\n",
    "            \"_source\": doc,\n",
    "        }\n",
    "    )\n",
    "\n",
    "# Use helpers.bulk to index\n",
    "helpers.bulk(client, documents)\n",
    "\n",
    "print(\"Done indexing documents into `openai-movie-embeddings` index!\")\n",
    "time.sleep(3)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cf0f6df7",
   "metadata": {},
   "source": [
    "## Semantic search\n",
    "\n",
    "After the dataset has been enriched with the embeddings, you can query the data using [semantic search](https://www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html#knn-semantic-search). Pass a `query_vector_builder` to the k-nearest neighbor (kNN) vector search API, and provide the query text and the model you have used to create the embeddings."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d9b21b71",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Score: 0.91674197\n",
      "Title: Fight Club\n",
      "Plot: An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.\n",
      "\n",
      "Score: 0.9069592\n",
      "Title: Pulp Fiction\n",
      "Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.\n",
      "\n",
      "Score: 0.8992071\n",
      "Title: The Dark Knight\n",
      "Plot: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.\n",
      "\n"
     ]
    }
   ],
   "source": [
    "response = client.search(\n",
    "    index=\"openai-movie-embeddings\",\n",
    "    size=3,\n",
    "    knn={\n",
    "        \"field\": \"plot_embedding\",\n",
    "        \"query_vector_builder\": {\n",
    "            \"text_embedding\": {\n",
    "                \"model_id\": \"my_openai_embedding_model\",\n",
    "                \"model_text\": \"Fighting movie\",\n",
    "            }\n",
    "        },\n",
    "        \"k\": 10,\n",
    "        \"num_candidates\": 100,\n",
    "    },\n",
    ")\n",
    "\n",
    "for hit in response[\"hits\"][\"hits\"]:\n",
    "    doc_id = hit[\"_id\"]\n",
    "    score = hit[\"_score\"]\n",
    "    title = hit[\"_source\"][\"title\"]\n",
    "    plot = hit[\"_source\"][\"plot\"]\n",
    "    print(f\"Score: {score}\\nTitle: {title}\\nPlot: {plot}\\n\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7e4055ba",
   "metadata": {},
   "source": [
    "**NOTE:** The value of `model_id` in the `query_vector_builder` must match the value of `inference_id` you created in the [first step](#create-the-inference-endpoint)."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3.12.3 64-bit",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.3"
  },
  "vscode": {
   "interpreter": {
    "hash": "b0fa6594d8f4cbf19f97940f81e996739fb7646882a419484c72d19e05852a7e"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}